In [1]:
import h5py
import numpy as np
from sklearn.preprocessing import StandardScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


In [3]:
# ef["layer_0"].astype('float32')
# type(ef["layer_0"].astype('float32')[1,1,1])

In [4]:
ef = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo/eplus.hdf5','r')
gf = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo/gamma.hdf5','r')
pf = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo/piplus.hdf5','r')

In [5]:
hdfs = [ef, gf, pf]
nplcats = []

for hdf in hdfs:
    npl0 = np.array(hdf['layer_0']).astype('float32')
    npl1 = np.array(hdf['layer_1']).astype('float32')
    npl2 = np.array(hdf['layer_2']).astype('float32')
    
    npl0 = npl0.reshape(npl0.shape[0], -1)
    npl1 = npl1.reshape(npl1.shape[0], -1)
    npl2 = npl2.reshape(npl2.shape[0], -1)
    
    nplcats.append(np.concatenate([npl0, npl1, npl2], axis=1))

In [6]:
nplcatscaled = []
transformers = []
arrmins = [[], [], []]
epsilon = 1e-2

for i in range(len(nplcats)):
    nparr = nplcats[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    transformer = StandardScaler().fit(nparr)
    nparr = transformer.transform(nparr)
    transformers.append(transformer)
    
    nparr = np.where(np.isnan(nparr), np.inf, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        
        if arrmin < 0 and not np.isnan(arrmin):
            nparr[:, j] -= arrmin
            nparr[:, j] += epsilon
            arrmins[i].append(arrmin)
        else:
            arrmins[i].append(0.)
            
    nparr = np.where(np.isinf(nparr), 0, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        if arrmin < 0:
            print(j, arrmin)
            
    nplcatscaled.append(nparr)

In [7]:
nplcats[2].shape
nplcatscaled[2].shape

(100000, 504)

In [8]:
ef_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo_scaled/eplus.hdf5','w')
gf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo_scaled/gamma.hdf5','w')
pf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/calo_scaled/piplus.hdf5','w')

In [9]:
hdfs_scaled = [ef_scaled, gf_scaled, pf_scaled]
layer_shapes = {}
for key in hdf.keys():
    if key == "energy" or key == "overflow":
        pass
    else:
        layer_shapes[key] = hdf[key].shape

In [10]:
layer_shapes

{'layer_0': (100000, 3, 96),
 'layer_1': (100000, 12, 12),
 'layer_2': (100000, 12, 6)}

In [11]:
for hdf, hdf_scaled, scaled_data in zip(hdfs, hdfs_scaled, nplcatscaled):
    offset = 0
    for key in hdf.keys():
        if key == "energy" or key == "overflow":
            hdf_scaled.create_dataset(key, data=hdf[key])
        else:
            layer_shape = layer_shapes[key]
            print(scaled_data.shape)
            layer_data = scaled_data[:, offset:offset+(layer_shape[1]*layer_shape[2])]
            print(layer_data.shape)
            layer_data = layer_data.reshape(layer_shape)
            hdf_scaled.create_dataset(key, data=layer_data)
            offset += layer_shape[1]*layer_shape[2]

(100000, 504)
(100000, 288)
(100000, 504)
(100000, 144)
(100000, 504)
(100000, 72)
(100000, 504)
(100000, 288)
(100000, 504)
(100000, 144)
(100000, 504)
(100000, 72)
(100000, 504)
(100000, 288)
(100000, 504)
(100000, 144)
(100000, 504)
(100000, 72)


In [12]:
for hdf_scaled in hdfs_scaled:
    for key in hdf_scaled.keys():
        print(key, hdf_scaled[key].shape, hdf_scaled[key].dtype)

energy (100000, 1) float64
layer_0 (100000, 3, 96) float32
layer_1 (100000, 12, 12) float32
layer_2 (100000, 12, 6) float32
overflow (100000, 3) float64
energy (100000, 1) float64
layer_0 (100000, 3, 96) float32
layer_1 (100000, 12, 12) float32
layer_2 (100000, 12, 6) float32
overflow (100000, 3) float64
energy (100000, 1) float64
layer_0 (100000, 3, 96) float32
layer_1 (100000, 12, 12) float32
layer_2 (100000, 12, 6) float32
overflow (100000, 3) float64


In [13]:
for hdf_scaled in hdfs_scaled:
    hdf_scaled.close()

In [14]:
for nplcat in nplcatscaled:
    print(nplcat.shape)

(100000, 504)
(100000, 504)
(100000, 504)


In [15]:
print(len(arrmins[0]))

504


In [16]:
np.sum(nplcatscaled[2] == np.nan)

0

In [17]:
nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isinf(nparr), 0, nparr)
    nplcatinv.append(nparr)

In [18]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if np.abs(diff) > 0:
            print(i, j, diff)

0 142 -0.3723908
0 143 0.5914335
0 144 1.3928509
0 353 9.338834
0 354 9.172087
0 365 -0.5450735
0 366 0.036711693
1 354 7.1371098
1 365 -9.951949
1 366 10.38685
2 144 -0.120242506


In [19]:
params = transformers[0].get_params()

In [20]:
params

{'copy': True, 'with_mean': True, 'with_std': True}

In [21]:
import joblib

In [22]:
joblib.dump(transformers[0], 'scaler.gz')
transformer = joblib.load('scaler.gz')

In [23]:
print(len(transformers))

3


In [24]:
joblib.dump(transformers[0], '/raid/javier/Datasets/CaloVAE/data/calo_scaled/eplus_scaler.gz')
joblib.dump(transformers[1], '/raid/javier/Datasets/CaloVAE/data/calo_scaled/gamma_scaler.gz')
joblib.dump(transformers[2], '/raid/javier/Datasets/CaloVAE/data/calo_scaled/piplus_scaler.gz')

['/raid/javier/Datasets/CaloVAE/data/calo_scaled/piplus_scaler.gz']

In [25]:
ld_transformers = []
ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/calo_scaled/eplus_scaler.gz'))
ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/calo_scaled/gamma_scaler.gz'))
ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/calo_scaled/piplus_scaler.gz'))

In [26]:
np.sum(nparr == np.inf)
# nplcatscaled[i]

0

In [27]:
nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.inf)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = ld_transformers[i]
    # try:
    nparr = transformer.inverse_transform(nparr)
    # except:
        # print(nparr)
    
    nparr = np.where(np.isnan(nparr), 0, nparr)
    nplcatinv.append(nparr)

In [28]:
import sklearn
print(sklearn.__version__)

0.24.2


In [29]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 1e-4:
            print(i, j, diff)

0 143 0.5914335
0 144 1.3928509
0 353 9.338834
0 354 9.172087
0 366 0.036711693
1 354 7.1371098
1 366 10.38685


In [38]:
print(len(arrmins))

3


In [30]:
# print(arrmins[2])

In [31]:
for i, ptype in enumerate(["eplus", "gamma", "piplus"]):
    filepath = "/raid/javier/Datasets/CaloVAE/data/calo_scaled/" + ptype + "_amin.npy"
    with open(filepath, 'wb') as f:
        np.save(f, arrmins[i])

In [37]:
# type(nplcatinv[0][1,1])

In [32]:
# ATLAS

In [33]:
gf = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas/photons_samples_highStat_En_5.hdf5', 'r')
pf = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas/pions_samples_highStat_En_5.hdf5', 'r')

In [34]:
hdfs = [gf,pf]
nplcats = []

for hdf in hdfs:
    npl0 = np.array(hdf['voxels'])
    
    npl0 = npl0.reshape(npl0.shape[0], -1)
    
    nplcats.append(np.concatenate([npl0], axis=1))
    
nplcatscaled = []
transformers = []
arrmins = [[], [], []]
epsilon = 1e-2

for i in range(len(nplcats)):
    nparr = nplcats[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    transformer = StandardScaler().fit(nparr)
    nparr = transformer.transform(nparr)
    transformers.append(transformer)
    
    nparr = np.where(np.isnan(nparr), np.inf, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        
        if arrmin < 0 and not np.isnan(arrmin):
            nparr[:, j] -= arrmin
            nparr[:, j] += epsilon
            arrmins[i].append(arrmin)
        else:
            arrmins[i].append(0.)
            
    nparr = np.where(np.isinf(nparr), 0, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        if arrmin < 0:
            print(j, arrmin)
            
    nplcatscaled.append(nparr)
    


In [35]:
gf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/photons_samples_highStat_En_5.hdf5', 'w')
pf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5.hdf5', 'w')


In [36]:

hdfs_scaled = [gf_scaled, pf_scaled]
# hdfs_scaled = [ef_scaled, gf_scaled, pf_scaled]
layer_shapes = {}
for hdf in hdfs:
    for key in hdf.keys():
        if key == "energy" or key == "overflow" or key == "energy_from_voxels":
            pass
        else:
#             layer_shapes[key] = hdf[key].shape
            layer_shapes[hdf] = {key : hdf[key].shape}
        
        

for hdf, hdf_scaled, scaled_data in zip(hdfs, hdfs_scaled, nplcatscaled):
    offset = 0
    for key in hdf.keys():
        if key == "energy" or key == "overflow" or key == "energy_from_voxels":
            hdf_scaled.create_dataset(key, data=hdf[key])
        else:
            layer_shape = layer_shapes[hdf][key]
            print("###########")
            print(layer_shape)
            print(scaled_data.shape)
#             layer_data = scaled_data[:, offset:offset+(layer_shape[1]*layer_shape[2])]
            layer_data = scaled_data[:, offset:offset+layer_shape[1]]
            print(layer_data.shape)
            layer_data = layer_data.reshape(layer_shape)
            hdf_scaled.create_dataset(key, data=layer_data)
#             offset += layer_shape[1]*layer_shape[2]
            offset += layer_shape[1]



###########
(99933, 368)
(99933, 368)
(99933, 368)
###########
(99999, 533)
(99999, 533)
(99999, 533)


In [37]:
for hdf_scaled in hdfs_scaled:
    for key in hdf_scaled.keys():
        print(key, hdf_scaled[key].shape, hdf_scaled[key].dtype)
        
for hdf_scaled in hdfs_scaled:
    hdf_scaled.close()
    
    


energy (99933, 1) float64
energy_from_voxels (99933, 1) float64
voxels (99933, 368) float64
energy (99999, 1) float64
energy_from_voxels (99999, 1) float64
voxels (99999, 533) float64


In [38]:
# Inverse transform


nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isinf(nparr), 0, nparr)
    nplcatinv.append(nparr)


for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 0:
            print(i, j, diff)
            
params = transformers[0].get_params()



In [ ]:
gf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/photons_samples_highStat_En_5.hdf5', 'w')
pf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5.hdf5', 'w')

In [39]:
import joblib

# joblib.dump(transformers[0], 'scaler.gz')
# transformer = joblib.load('scaler.gz')

joblib.dump(transformers[0], '/raid/javier/Datasets/CaloVAE/data/atlas_scaled/photons_samples_highStat_En_5_scaler.gz')
joblib.dump(transformers[1], '/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5_scaler.gz')



['/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5_scaler.gz']

In [40]:
ld_transformers = []
ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/photons_samples_highStat_En_5_scaler.gz'))
ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5_scaler.gz'))

nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.inf)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = ld_transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isnan(nparr), 0, nparr)
    nplcatinv.append(nparr)
    


In [41]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 1e-4:
            print(i, j, diff)
            


In [42]:
for i, ptype in enumerate(["photons_samples_highStat_En_5", "pions_samples_highStat_En_5"]):
    filepath = "/raid/javier/Datasets/CaloVAE/data/atlas_scaled/" + ptype + "_amin.npy"
    with open(filepath, 'wb') as f:
        np.save(f, arrmins[i])

In [ ]:
# ATLAS ds 2

In [4]:
gf = h5py.File("/fast_scratch/QVAE/data/atlas_dataset2and3_uniform/dataset_2_1.hdf5", 'r')

In [5]:
hdfs = [gf]
nplcats = []

for hdf in hdfs:
    npl0 = np.array(hdf['showers'])
    
    npl0 = npl0.reshape(npl0.shape[0], -1)
    
    nplcats.append(np.concatenate([npl0], axis=1))
    
nplcatscaled = []
transformers = []
arrmins = [[], [], []]
epsilon = 1e-2

for i in range(len(nplcats)):
    nparr = nplcats[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    transformer = StandardScaler().fit(nparr)
    nparr = transformer.transform(nparr)
    transformers.append(transformer)
    
    nparr = np.where(np.isnan(nparr), np.inf, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        
        if arrmin < 0 and not np.isnan(arrmin):
            nparr[:, j] -= arrmin
            nparr[:, j] += epsilon
            arrmins[i].append(arrmin)
        else:
            arrmins[i].append(0.)
            
    nparr = np.where(np.isinf(nparr), 0, nparr)
    
    for j in range(nparr.shape[1]):
        arrmin = np.amin(nparr[:, j])
        if arrmin < 0:
            print(j, arrmin)
            
    nplcatscaled.append(nparr)
    


In [6]:
gf_scaled = h5py.File("/fast_scratch/QVAE/data/atlas_dataset2and3_scaled/uniform_dataset_2_1.hdf5", 'w')
# pf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5.hdf5', 'w')


In [7]:

hdfs_scaled = [gf_scaled]
# hdfs_scaled = [ef_scaled, gf_scaled, pf_scaled]
layer_shapes = {}
for hdf in hdfs:
    for key in hdf.keys():
        if key == "energy" or key == "overflow" or key == "energy_from_voxels":
            pass
        else:
#             layer_shapes[key] = hdf[key].shape
            layer_shapes[hdf] = {key : hdf[key].shape}
        
        

for hdf, hdf_scaled, scaled_data in zip(hdfs, hdfs_scaled, nplcatscaled):
    offset = 0
    for key in hdf.keys():
        if key == "energy" or key == "overflow" or key == "energy_from_voxels" or key == "incident_energies":
            hdf_scaled.create_dataset(key, data=hdf[key])
        else:
            layer_shape = layer_shapes[hdf][key]
            print("###########")
            print(layer_shape)
            print(scaled_data.shape)
#             layer_data = scaled_data[:, offset:offset+(layer_shape[1]*layer_shape[2])]
            layer_data = scaled_data[:, offset:offset+layer_shape[1]]
            print(layer_data.shape)
            layer_data = layer_data.reshape(layer_shape)
            hdf_scaled.create_dataset(key, data=layer_data)
#             offset += layer_shape[1]*layer_shape[2]
            offset += layer_shape[1]



###########
(15924, 6480)
(15924, 6480)
(15924, 6480)


In [8]:
for hdf_scaled in hdfs_scaled:
    for key in hdf_scaled.keys():
        print(key, hdf_scaled[key].shape, hdf_scaled[key].dtype)
        
for hdf_scaled in hdfs_scaled:
    hdf_scaled.close()
    
    


incident_energies (15924, 1) float64
showers (15924, 6480) float64


In [9]:
# Inverse transform


nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    nparr = np.where(nparr > 0., nparr, np.nan)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    nparr = np.where(np.isinf(nparr), 0, nparr)
    nplcatinv.append(nparr)


for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 0:
            print(i, j, diff)
            
params = transformers[0].get_params()



In [10]:
# gf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_dataset2and3_scaled/dataset_2_1.hdf5', 'w')
# pf_scaled = h5py.File('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5.hdf5', 'w')

In [11]:
import joblib

# joblib.dump(transformers[0], 'scaler.gz')
# transformer = joblib.load('scaler.gz')

joblib.dump(transformers[0], '/fast_scratch/QVAE/data/atlas_dataset2and3_scaled/uniform_dataset_2_1_scaler.gz')
# joblib.dump(transformers[1], '/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5_scaler.gz')



['/fast_scratch/QVAE/data/atlas_dataset2and3_scaled/uniform_dataset_2_1_scaler.gz']

In [12]:
ld_transformers = []
ld_transformers.append(joblib.load('/fast_scratch/QVAE/data/atlas_dataset2and3_scaled/uniform_dataset_2_1_scaler.gz'))
# ld_transformers.append(joblib.load('/raid/javier/Datasets/CaloVAE/data/atlas_scaled/pions_samples_highStat_En_5_scaler.gz'))

nplcatinv = []

for i in range(len(nplcatscaled)):
    nparr = nplcatscaled[i]
    # nparr = np.where(nparr > 0., nparr, np.inf)
    nparr = np.where(nparr > 0., nparr, np.nan)
    
    for j in range(nparr.shape[1]):
        arrmin = arrmins[i][j]
        if arrmin < 0. and not np.isnan(arrmin):
            nparr[:, j] -= epsilon
            nparr[:, j] += arrmin
            
    transformer = ld_transformers[i]
    nparr = transformer.inverse_transform(nparr)
    
    # nparr = np.where(np.isnan(nparr), 0, nparr)
    nparr = np.where(np.isnan(nparr), 0, nparr)
    nplcatinv.append(nparr)
    


In [13]:
for i in range(len(nplcatinv)):
    nparrorig = nplcats[i]
    nparrinv = nplcatinv[i]
    
    for j in range(nparrorig.shape[1]):
        diff = np.sum(nparrorig[:, j] - nparrinv[:, j])
        if diff > 1e-4:
            print(i, j, diff)
            


In [14]:
for i, ptype in enumerate(["dataset_2_1"]):
    filepath = "/fast_scratch/QVAE/data/atlas_dataset2and3_scaled/uniform_" + ptype + "_amin.npy"
    with open(filepath, 'wb') as f:
        np.save(f, arrmins[i])